In [1]:
# torch.autograd 를 사용한 자동 미분
# 신경망 학습에는 역전파 알고리즘이 주로 사용됨, 손실 함수의 gradient에 따라 조정됨.
# gradient를 계산하기 위한 자동 미분 엔진이 바로 torch.autograd

import torch

x = torch.ones(5)  # input tensor
y = torch.zeros(3) # expected output
w = torch.randn(5, 3, requires_grad=True)   #requires_grad 속성이 True로 설정되어야 grad 속성 구할 수 있음.
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w)+b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

In [2]:
# 5-1. Tensor, Funcion과 연산그래프(Computational graph)
print('Gradient function for z =', z.grad_fn)
print('Gradient function for loss =', loss.grad_fn)

Gradient function for z = <AddBackward0 object at 0x7f70c8701c90>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward object at 0x7f70c8701d50>


In [3]:
# 5-2. 변화도(Gradient) 계산하기
# 신경망에서 매개변수의 가중치w를 최적화하려면 매개변수(w,b)에 대한 손실함수의 도함수(derivative)를 계산해야 한다.
# 도함수 계산하기 위해 loss.backward()를 호출한 다음 w.grad와 b.grad에서 값을 가져온다.
loss.backward()
print(w.grad)
print(b.grad)

tensor([[0.2736, 0.0554, 0.0790],
        [0.2736, 0.0554, 0.0790],
        [0.2736, 0.0554, 0.0790],
        [0.2736, 0.0554, 0.0790],
        [0.2736, 0.0554, 0.0790]])
tensor([0.2736, 0.0554, 0.0790])


In [4]:
# 5-3. 변화도 추적 멈추기
# 순전파 연산만 필요한 경우(모델을 학습한 뒤 데이터를 단순히 적용하는 경우), 추적이나 지원이 필요없을 수 있다.
z = torch.matmul(x, w)+b
print(z.requires_grad)

with torch.no_grad():  #연산 추적 멈춤
    z = torch.matmul(x, w)+b
print(z.requires_grad)

True
False


In [5]:
# 위와 동일한 방법
z = torch.matmul(x, w)+b
z_det = z.detach()
print(z_det.requires_grad)

False


In [7]:
# 5-5. 선택적으로 읽기(Optional Reading): 텐서 변화도와 야코비안 곱(Jacobian Product)
# 출력함수가 임의의 텐서인 경우, 실제 gradient가 아닌 야코비안 곱을 계산한다.
# v를 인자로 backward를 호출. v의 크기는 곱을 계산하려고 하는 원래 텐서의 크기와 같아야 함.
inp = torch.eye(5, requires_grad=True)
out = (inp+1).pow(2)
out.backward(torch.ones_like(inp), retain_graph=True)
print("First call\n", inp.grad)
out.backward(torch.ones_like(inp), retain_graph=True)
print("\nSecond call\n", inp.grad)
inp.grad.zero_()
out.backward(torch.ones_like(inp), retain_graph=True)
print("\nCall after zeroing gradients\n", inp.grad)

First call
 tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.],
        [2., 2., 2., 2., 4.]])

Second call
 tensor([[8., 4., 4., 4., 4.],
        [4., 8., 4., 4., 4.],
        [4., 4., 8., 4., 4.],
        [4., 4., 4., 8., 4.],
        [4., 4., 4., 4., 8.]])

Call after zeroing gradients
 tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.],
        [2., 2., 2., 2., 4.]])
